In [1]:
import sys
sys.path.append('../')


In [155]:
import pandas as pd
import pickle
from tqdm import tqdm

In [2]:
from curry.km import DBPediaPhysicsResources

In [3]:
phy = DBPediaPhysicsResources()

In [7]:
babel = phy.relevant_annotations(annotation_source='babel')

In [8]:
wikify = phy.relevant_annotations(annotation_source='wikifier')

In [9]:
len(babel)

539

In [10]:
len(wikify)

539

In [17]:
pd.Series([r for url in babel for r in babel[url]]).value_counts().sort_values(ascending=False)

http://dbpedia.org/resource/Thermodynamic_free_energy      452
http://dbpedia.org/resource/Force                          410
http://dbpedia.org/resource/Physical_body                  390
http://dbpedia.org/resource/Light                          349
http://dbpedia.org/resource/Electric_charge                318
                                                          ... 
http://dbpedia.org/resource/Annotation                       1
http://dbpedia.org/resource/Enzyme                           1
http://dbpedia.org/resource/Cereal                           1
http://dbpedia.org/resource/Motte-and-bailey_castle          1
http://dbpedia.org/resource/Cost-effectiveness_analysis      1
Length: 2852, dtype: int64

## Get DBPedia Resources for Elixier

In [18]:
elixier = pd.read_csv('../data/elixier_physics_all.csv', index_col=0)

In [22]:
with open('../data/elixier_physics_all_responses.pkl', 'rb') as f:
    elixier_responses = pickle.load(f)

In [24]:
success_mask = pd.Series(elixier_responses).\
    apply(lambda r: (not isinstance(r, Exception)) and
          (r.status_code == 200))
elixier = elixier[success_mask]

In [25]:
text_columns = ['description',
                'titel für schüler', 
                'lange beschreibung', 
                'beschreibung für schüler', 
                'schlagwörter', 
                'freie schlagwörter']

In [96]:
def clean_description(desc):
    if 'Lernressourcentyp' in desc:
        return desc.split('; Lernressourcentyp')[0]
    elif 'margin-' in desc: 
        return None
    else:
        return desc

In [107]:
descriptions = elixier.description[elixier.description.notna()].apply(clean_description)
descriptions = descriptions[descriptions.notna()]
descriptions = descriptions[descriptions.apply(lambda x: len(x.strip())) >= 2]

In [121]:
to_babelfy = []
group = []
mappings = {}

for description_index in descriptions.index:
    desc = descriptions.loc[description_index]
    group.append((description_index, desc))
    if len(group) == 3:
        joined_index = len(to_babelfy)
        out = ''
        for desc_index, desc in group:
            start = len(out)
            out += desc
            out += ' '
            end = len(out)
            mappings[desc_index] = (joined_index, start, end)
        to_babelfy.append(out)
        group = []
        
joined_index = len(to_babelfy)
out = ''
for desc_index, desc in group:
    start = len(out)
    out += desc
    out += '. '
    end = len(out)
    mappings[desc_index] = (joined_index, start, end)
to_babelfy.append(out)
group = []


In [122]:
len(mappings)

2705

In [123]:
descriptions.shape

(2705,)

In [124]:
len(to_babelfy)

902

In [126]:
from curry.babelfy import Babelfier

In [127]:
babelfier = Babelfier(cache_dir='../data/cache/')

In [129]:
sample_out = babelfier.bab(to_babelfy[0])

In [139]:
s[sample_out[1]['start']: sample_out[1]['end'] + 1]

'August 2009'

In [154]:
for desc_id in mappings:
    group_id, start, end = mappings[desc_id]
    desc = to_babelfy[group_id][start: end]
    print(descriptions.loc[desc_id])
    print(desc)
    print(desc.strip() == descriptions.loc[desc_id].strip())
    print('-' * 40)

Am 17. August 2009 steht der Planet Neptun im Sternbild Steinbock in Opposition zur Sonne.
Am 17. August 2009 steht der Planet Neptun im Sternbild Steinbock in Opposition zur Sonne. 
True
----------------------------------------
Mit XEphem können Sie den Transit oder andere Konstellationen im Unterricht vor- und nachbereiten (je nach Vertiefung alle Jahrgänge und Schulformen).
Mit XEphem können Sie den Transit oder andere Konstellationen im Unterricht vor- und nachbereiten (je nach Vertiefung alle Jahrgänge und Schulformen). 
True
----------------------------------------
Viele interaktive Flash-Applikationen der Physikdidaktik der Uni Bayreuth aus dem Bereich der Elektrizitätslehre zum Einsatz in der Mittelstufe.
Viele interaktive Flash-Applikationen der Physikdidaktik der Uni Bayreuth aus dem Bereich der Elektrizitätslehre zum Einsatz in der Mittelstufe. 
True
----------------------------------------
Eine Software zur Simulation einer Wellenwanne und Java-Applets aus dem Internet stel

Die gebundene Rotation Joachim Herz Stiftung Abb. 1 Die Gezeitenkräfte der Erde auf dem Mond bewirken das sich die Rotation des Mondes an die Umlaufzeit um die Erde angepasst hat. Der Einfluss der Gezeitenkräfte der Erde
Die gebundene Rotation Joachim Herz Stiftung Abb. 1 Die Gezeitenkräfte der Erde auf dem Mond bewirken das sich die Rotation des Mondes an die Umlaufzeit um die Erde angepasst hat. Der Einfluss der Gezeitenkräfte der Erde 
True
----------------------------------------
Über phyphox Die App phyphox wird von der RWTH Aachen entwickelt und steht allen Interessierten kostenlos zur Verfügung. phyphox ermöglicht es dir, mit den Sensoren deines Smartphones zu experimentieren, Messwerte aufzunehmen und auszuwerten. Hier geht es zur Website des Projektes /
Über phyphox Die App phyphox wird von der RWTH Aachen entwickelt und steht allen Interessierten kostenlos zur Verfügung. phyphox ermöglicht es dir, mit den Sensoren deines Smartphones zu experimentieren, Messwerte aufzunehmen u

True
----------------------------------------
Menschen nutzen seit vielen Jahrhunderten die Wasserkraft als Antrieb. Dabei schlug das Wasser entweder von unten oder von oben auf das Wasserrad. ... Diese Aufgabe dient der Entwicklung folgender Kompetenzen: Die Schülerinnen und Schüler können - unter Anleitung Recherchen zu technischen Anwendungen durchführen, - Bewegungsvorgänge in Natur und Technik energetisch beschreiben, - Energiebilanzen aufstellen und - Wirkungsgrade berechnen, - alternative technische Lösungen nach vorgegebenen Kriterien vergleichen.
Menschen nutzen seit vielen Jahrhunderten die Wasserkraft als Antrieb. Dabei schlug das Wasser entweder von unten oder von oben auf das Wasserrad. ... Diese Aufgabe dient der Entwicklung folgender Kompetenzen: Die Schülerinnen und Schüler können - unter Anleitung Recherchen zu technischen Anwendungen durchführen, - Bewegungsvorgänge in Natur und Technik energetisch beschreiben, - Energiebilanzen aufstellen und - Wirkungsgrade berechne

----------------------------------------
Das digital.learning.lab ist ein Hamburger Kompetenzzentrum für die Unterrichtsgestaltung in digitalen Zeiten. Lehrkräfte finden hier Anregungen und Unterstützung, ihren Unterricht unter Berücksichtigung der Kompetenzen für eine digitalisierte Lebens- und Arbeitswelt weiter zu entwickeln. Die Ideen zur Unterrichtsgestaltung sind den jeweiligen Kompetenzbereichen zugeordnet. Durch einen Filter lassen sich Themen für die Klassen der Primarstufen sowie der Sek I und Sek II eingrenzen. Auch Lehrkräfte anderer Bundesländer können sich auf der Plattform Anregungen für ihren Unterricht holen.
Das digital.learning.lab ist ein Hamburger Kompetenzzentrum für die Unterrichtsgestaltung in digitalen Zeiten. Lehrkräfte finden hier Anregungen und Unterstützung, ihren Unterricht unter Berücksichtigung der Kompetenzen für eine digitalisierte Lebens- und Arbeitswelt weiter zu entwickeln. Die Ideen zur Unterrichtsgestaltung sind den jeweiligen Kompetenzbereichen z

In [156]:
annotations = []

for b in tqdm(to_babelfy):
    annotations.append(babelfier.bab(b))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 902/902 [11:10<00:00,  1.35it/s]


In [157]:
with open('../data/elixier_babelfy_annotations.pkl', 'wb') as f:
    pickle.dump(annotations, f)

In [158]:

len(annotations)

902

In [188]:
desc_id_to_annotation = dict()
for desc_id in mappings:
    desc_id_to_annotation[desc_id] = []
    group_id, start, end = mappings[desc_id]
    desc = to_babelfy[group_id][start: end]
    for ann in annotations[group_id]:
        ann_start, ann_end = ann['start'], ann['end']
        if (ann_start >= start) and (ann_end <= end):
            desc_id_to_annotation[desc_id].append(ann)

In [189]:
errors = []
desc_to_annotation = dict()
for desc_id in tqdm(desc_id_to_annotation):
    corrected = []
    for ann in desc_id_to_annotation[desc_id]:
        if ann['text'] not in descriptions.loc[desc_id]:
            errors.append(f'{ann["text"]} not in {descriptions.loc[desc_id]}')
        else:
            corrected.append(ann)
    desc_to_annotation[descriptions.loc[desc_id]] = corrected
    desc_id_to_annotation[desc_id] = corrected

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 2705/2705 [00:00<00:00, 5900.09it/s]


In [190]:

errors

['Karlheinz not in Tipps zum kreativ werden! \xa0\xa0\xa0 Mehlstaub macht Lichtwege sichtbar. \xa0\xa0\xa0 Seifenblasen schillern bunt. \xa0\xa0\xa0 Spielzeugfiguren stellen kleine Menschen dar. \xa0\xa0\xa0 Tinte färbt Wasser.',
 'Karlheinz Meier not in Tipps zum kreativ werden! \xa0\xa0\xa0 Mehlstaub macht Lichtwege sichtbar. \xa0\xa0\xa0 Seifenblasen schillern bunt. \xa0\xa0\xa0 Spielzeugfiguren stellen kleine Menschen dar. \xa0\xa0\xa0 Tinte färbt Wasser.',
 'Meier not in Tipps zum kreativ werden! \xa0\xa0\xa0 Mehlstaub macht Lichtwege sichtbar. \xa0\xa0\xa0 Seifenblasen schillern bunt. \xa0\xa0\xa0 Spielzeugfiguren stellen kleine Menschen dar. \xa0\xa0\xa0 Tinte färbt Wasser.',
 'Versuch not in Karlheinz Meier von der Universität Heidelberg stellt leicht verständliche Videos zum Physikunterricht zur Verfügung. In anderthalb Minuten wird gut fassbar in das Prinzip einer technischen Erfindung eingeführt oder ein physikalisches Phä',
 'Aufgabe not in Medium 1 \xa0 1. Brechungsindex: 

In [191]:
with open('../data/elixier_desc_to_annotation.pkl', 'wb') as f:
    pickle.dump(desc_to_annotation, f)
    
with open('../data/elixier_id_to_annotation.pkl', 'wb') as f:
    pickle.dump(desc_id_to_annotation, f)

In [219]:
import sparql_dataframe
from functools import lru_cache

def get_rdf_type(resource, depth):
    endpoint = "http://dbpedia.org/sparql"

    q = """
        prefix dbr: <http://dbpedia.org/resource/> 
        prefix dbo: <http://dbpedia.org/ontology/>
        prefix skos: <http://www.w3.org/2004/02/skos/core#>

        select distinct ?subcategory  where {
          <""" + resource + """> dbo:wikiPageRedirects* ?redirect.
          ?redirect <http://purl.org/dc/terms/subject> ?category.
          ?category """ + '?/'.join(['skos:broader' for _ in range(depth)]) + """ ?subcategory
        }
    """
    df = sparql_dataframe.get(endpoint, q)
    return df.subcategory.values

In [196]:
entities = pd.Series([ann.get('DBpediaURL') for anns in desc_id_to_annotation.values() for ann in anns]).value_counts()

In [ ]:
depth_to_entity_to_categories = dict()
for depth in [2,3,4]:
    entity_to_categories = dict()
    for e in tqdm(entities[entities.index != ''].index):
        entity_to_categories[e] = get_rdf_type(e, depth)
    depth_to_entity_to_categories[depth] = entity_to_categories

 79%|███████████████████████████████████████████████████████████████████████████████▍                    | 3276/4126 [17:08<03:54,  3.62it/s]

In [222]:
depth_to_entity_to_categories[5] = entity_to_categories

In [228]:
with open('../data/elixier_annotations_categories.pkl', 'rb') as f:
    depth_to_entity_to_categories = pickle.load(f)

In [230]:
for depth in depth_to_entity_to_categories:
    categories = pd.Series([c for cs in depth_to_entity_to_categories[depth].values() for c in cs])
    pct = categories[categories.apply(phy.is_accepted_topics)].shape[0] / categories.shape[0]
    print(pct)

0.017668542098509526
0.017668542098509526
0.017668542098509526
0.017668542098509526


In [231]:

filtered_categories = {e: [cat for cat in entity_to_categories[e] if phy.is_accepted_topics(cat)] 
                       for e in entity_to_categories}

In [215]:
import numpy as np

In [216]:
np.mean(list(map(len, entity_to_categories.values())))

287.28235579253516